In [2]:
import pandas as pd

#Abre os bancos de dados necessários
pop = pd.read_csv('base_csv/pop2020.csv')
io = pd.read_csv('base_csv/caso_full.csv')


In [3]:
#Arruma os nomes das colunas para realizar o merge
pop.rename(columns={'Total':'total_inhabitants','Armenor':'ibge_code'}, inplace = True)

In [4]:
#Filtra somente os dados do último dia das semanas epidemiológicas
dates = ['2020-02-29', '2020-03-07', '2020-03-14', '2020-03-21',
               '2020-03-28', '2020-04-04', '2020-04-11', '2020-04-18',
               '2020-04-25', '2020-05-02', '2020-05-09', '2020-05-16',
               '2020-05-23', '2020-05-30', '2020-06-06', '2020-06-13',
               '2020-06-20', '2020-06-27', '2020-07-04', '2020-07-11',
               '2020-07-18', '2020-07-25', '2020-08-01']

datesDF = pd.DataFrame(dates,columns=['date'])

select_for_dates = pd.merge(io,datesDF, how = 'inner')

#Arruma o nome das colunas
select_for_dates.rename(columns={'city_ibge_code':'ibge_code','date':'week_last_day'}, inplace = True)


In [5]:
#Classifica os estados por macro regiões
def region(x):
    regions = {
        'norte':['AM','RR','AP','PA','TO','RO','AC'],
        'nordeste':['MA','PI','CE','RN','PE','PB','SE','AL','BA'],
        'centro-oeste':['MT','MS','GO','DF'],
        'sudeste':['MG','SP','RJ','ES'],
        'sul':['PR','RS','SC']       
    }
    for k,v in regions.items():
        if x['state'] in v:
            return k
    
select_for_dates['region'] = select_for_dates.apply(region,axis=1)

In [6]:
#Seleciona o df de população para cálculo por cidades
pop_cities = pop[['ibge_code','total_inhabitants']].copy()

In [7]:
#Coloca o total da população das cidades no df selecionado por datas
cities_df = pd.merge(select_for_dates,pop_cities, how = 'inner')


In [8]:
#Encontrar o valor dos novos casoso por semana
cities_df = cities_df.sort_values(by=['ibge_code','week_last_day']).copy()

last_city = 0
last_num = 0 


    
def get_num(x):
    global last_num, last_city
    if last_city == x['ibge_code']:
        current = x['last_available_confirmed']-last_num
        last_num = x['last_available_confirmed']
        return current
    else:
        last_city = x['ibge_code']
        last_num = 0
        return get_num(x)



cities_df['new_week_cases'] = cities_df.apply(get_num , axis=1)


In [9]:
#Encontrar o valor dos novos óbitos por semana
last_city = 0
last_num = 0 


    
def get_num(x):
    global last_num, last_city
    if last_city == x['ibge_code']:
        current = x['last_available_deaths']-last_num
        last_num = x['last_available_deaths']
        return current
    else:
        last_city = x['ibge_code']
        last_num = 0
        return get_num(x)



cities_df['new_week_deaths'] = cities_df.apply(get_num , axis=1)


In [10]:
#DF por cidades
cities_df= cities_df.copy().reset_index()
cities_df = cities_df[['region','state','city','ibge_code','epidemiological_week','total_inhabitants','last_available_confirmed','last_available_deaths','new_week_cases','new_week_deaths']].copy()

In [11]:
#DF por estado
states_df = cities_df.groupby(['state','epidemiological_week']).sum().reset_index()
states_df = states_df[['state','epidemiological_week','total_inhabitants','last_available_confirmed','last_available_deaths','new_week_cases','new_week_deaths']].copy()

In [12]:
#DF por macro região 
region_df = cities_df.groupby(['region','epidemiological_week']).sum().reset_index()
region_df = region_df[['region','epidemiological_week','total_inhabitants','last_available_confirmed','last_available_deaths','new_week_cases','new_week_deaths']].copy()

In [13]:
#DF Brasil
brazil_df = states_df.groupby(['epidemiological_week']).sum().reset_index()
brazil_df = brazil_df[['epidemiological_week','total_inhabitants','last_available_confirmed','last_available_deaths','new_week_cases','new_week_deaths']].copy()

In [14]:
#---------------------Indicadores ---------------------

In [15]:
#Indicador de Casos e Óbitos acumulados,por semana epidemiológica, já esta no DF, 
#respesctivamente nas colunas last_available_confirmed e last_available_deaths

In [16]:
#Novos Casos e Novos Óbitos, por semana epidemiológica, já está no DF,
#respesctivamente nas colunas new_week_cases e new_week_deaths

In [17]:
def indicators(data_frame):
    #Prevalência
    data_frame['prevalence'] = data_frame.apply(lambda x:x['last_available_confirmed']*100000/x['total_inhabitants'] , axis=1)
    
    #Indicador Mortalidade
    data_frame['mortality'] = data_frame.apply(lambda x:x['last_available_deaths']*100000/x['total_inhabitants'] , axis=1)
    
    #Indicador de Letalidade
    def lethality(x):
        if x['last_available_confirmed']==0:
            return 0
        else:
            return x['last_available_deaths']/x['last_available_confirmed']
    data_frame['lethality'] = data_frame.apply(lethality, axis=1)
    
    #Incidência de novos casos
    data_frame['incidence_cases'] = data_frame.apply(lambda x:x['new_week_cases']*100000/x['total_inhabitants'] , axis=1)
    
    #Incidência de novos óbitos
    data_frame['incidence_deaths'] = data_frame.apply(lambda x:x['new_week_deaths']*100000/x['total_inhabitants'] , axis=1)

    #Indicador Fator de Crescimento  dos casos
    last_number_cases=0

    def get_FC_cases(x):
        nonlocal last_number_cases
        if last_number_cases ==0:
            last_number_cases = x['new_week_cases']
            return 0
        else:
            fc= x['new_week_cases']/last_num
            last_number_cases = x['new_week_cases']
            return fc

    data_frame['cases_growth_factor'] = data_frame.apply(get_FC_cases, axis=1)

    #Indicador Fator de Crescimento  dos óbitos
    last_number_deaths=0

    def get_FC_deaths(x):
        nonlocal last_number_deaths
        if last_number_deaths ==0:
            last_number_deaths = x['new_week_deaths']
            return 0
        else:
            fc= x['new_week_deaths']/last_num
            last_number = x['new_week_deaths']
            return fc

    data_frame['deaths_growth_factor'] = data_frame.apply(get_FC_deaths, axis=1)

In [18]:
#Cálcula os indicadores para todos os DF
indicators(cities_df)
indicators(states_df)
indicators(region_df)
indicators(brazil_df)

In [19]:
cities_df.to_csv('resultados/cities_df.csv')

In [20]:
states_df.to_csv('resultados/states_df.csv')

In [21]:
region_df.to_csv('resultados/region_df.csv')

In [23]:
brazil_df.to_csv('resultados/brazil_df.csv')